<a href="https://colab.research.google.com/github/norgaston/laboratorio3-mlbd/blob/main/Laboratorio_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Laboratorio 3 - Clasificador zonas según el peligro**

**ELEMENTOS DE APRENDIZAJE DE MÁQUINA Y BIG DATA**

Carrera:  T.U. TECNOLOGÍAS DE PROGRAMACIÓN SEDE PUNTA ALTA

Facultad: Facultad de la Micro, Pequeña y Mediana Empresa (UPSO)

Docente: Valentín Barco

Cuatrimestre/Año: 2° Cuatrimestre 2023



En el laboratorio, nos centraremos en un conjunto de datos que incluye registros de
crímenes de 1973 en distintos estados de EE. UU., así como el porcentaje de la población
que reside en zonas urbanas en esos estados. El propósito es clasificar los estados en
diferentes categorías basadas en estos factores.



### **Conjunto de datos**

**Crímenes**

● **Asesinatos:** La tasa de asesinatos, que se suele calcular como el número de
asesinatos por cada 100,000 habitantes en el estado.

● **Asaltos:** La tasa de asaltos, que también se calcula por cada 100,000 habitantes.
Esto incluiría casos de ataques violentos, incluyendo agresiones físicas.

● **Población Urbana:** El porcentaje de la población que vive en áreas urbanas del
estado. Un número más alto indica una mayor concentración de población en las
ciudades.

● **Violaciónes:** La tasa de violaciones, similarmente calculada por cada 100,000
habitantes. Representa la incidencia de este tipo de crimen en el estado

### **Tarea a desarrollar**

Se debe desarrollar un programa que permita clasificar los estados en diferentes categorías,
dividiéndolos en zonas de baja peligrosidad y alta peligrosidad. Este proyecto debe ser
ejecutado en un Notebook de Colab.



## **Bibliotecas necesarias**

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go

## **Carga y análisis exploratorio de los datos**

In [2]:
# Cargar el conjunto de datos
ruta = '/content/Crimenes.csv'  # Ruta desde donde voy a cargar el dataset
data = pd.read_csv(ruta, encoding='latin-1')

In [3]:
data.head()

,Estado,Asesinatos,Asaltos,Poblacion Urbana,Violaciones
0,Alabama,13.2,236,58,21.2
1,Alaska,10.0,263,48,44.5
2,Arizona,8.1,294,80,31.0
3,Arkansas,8.8,190,50,19.5
4,California,9.0,276,91,40.6


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Estado            50 non-null     object 
 1   Asesinatos        50 non-null     float64
 2   Asaltos           50 non-null     int64  
 3   Poblacion Urbana  50 non-null     int64  
 4   Violaciones       50 non-null     float64
dtypes: float64(2), int64(2), object(1)
memory usage: 2.1+ KB


In [5]:
data.describe()

,Asesinatos,Asaltos,Poblacion Urbana,Violaciones
count,50.00000,50.000000,50.000000,50.000000
mean,7.78800,170.760000,65.540000,21.232000
std,4.35551,83.337661,14.474763,9.366385
min,0.80000,45.000000,32.000000,7.300000
25%,4.07500,109.000000,54.500000,15.075000
50%,7.25000,159.000000,66.000000,20.100000
75%,11.25000,249.000000,77.750000,26.175000
max,17.40000,337.000000,91.000000,46.000000


In [6]:
data.isnull().sum()

Estado              0
Asesinatos          0
Asaltos             0
Poblacion Urbana    0
Violaciones         0
dtype: int64

## **Preparado de los datos**

In [7]:
# Normalizar las características (escalar las características para que tengan una media de cero y una desviación estándar de uno)
scaler = StandardScaler()
data[['Asesinatos', 'Asaltos', 'Violaciones']] = scaler.fit_transform(data[['Asesinatos', 'Asaltos', 'Violaciones']])

## **Búsqueda del número óptimo de clústeres**

In [8]:
# Calcular la inercia para diferentes cantidades de clústeres
inertia = []
for n_clusters in range(1, 11):
    kmeans = KMeans(n_clusters=n_clusters, random_state=0, n_init=10)
    kmeans.fit(data[['Asesinatos', 'Asaltos', 'Violaciones']])
    inertia.append(kmeans.inertia_)

# Crear el gráfico interactivo con Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(1, 11)), y=inertia, mode='lines+markers'))
fig.update_layout(title='Curva del Codo',
                  xaxis_title='Número de Clústeres',
                  yaxis_title='Inercia')
fig.show()


## **Creación del modelo de clasificación**

In [9]:
# Crear un modelo K-Means con 3 clústeres (zonas de baja peligrosidad, media peligrosidad y alta peligrosidad)
kmeans = KMeans(n_clusters=3, random_state=0, n_init=10)

# Aplicar K-Means a los datos
data['Cluster'] = kmeans.fit_predict(data[['Asesinatos', 'Asaltos', 'Violaciones']])


## **Gráfico de los resultados**

In [10]:
# Asignar colores a los clústeres (0: verde, 1: amarillo, 2: rojo)
data['Color'] = data['Cluster'].map({0: '#9EFFA3', 1: '#FFFFB8', 2: '#FF9E9E'})

# Ordenar los estados por población
data = data.sort_values(by='Poblacion Urbana', ascending=True)

# Crear un gráfico de barras agrupadas con Plotly Graph Objects
fig = go.Figure()

# Agregar una barra para cada estado
for _, row in data.iterrows():
    fig.add_trace(go.Bar(
        x=[row['Estado']],
        y=[row['Poblacion Urbana']],
        marker=dict(color=row['Color']),
        showlegend=False,
    ))

# Configurar el diseño del gráfico de barras
fig.update_layout(
    title='Clasificación de Estados por Peligrosidad',
    xaxis_title='Estado',
    yaxis_title='Población Urbana',
)

# Agregar leyenda
legend_items = [
    go.Scatter(x=[None], y=[None], marker=dict(color='#9EFFA3'), name='Baja Peligrosidad'),
    go.Scatter(x=[None], y=[None], marker=dict(color='#FFFFB8'), name='Mediana Peligrosidad'),
    go.Scatter(x=[None], y=[None], marker=dict(color='#FF9E9E'), name='Alta Peligrosidad'),
]
for item in legend_items:
    fig.add_trace(item)

fig.show()


## **Análisis de Clasificación de Estados por Peligrosidad**

### **Resumen**

El modelo de K-Means ha agrupado los estados en tres clusters, representados por colores distintos. Los clusters se asocian con niveles de peligrosidad: verde para baja, amarillo para media y rojo para alta. A continuación, se presentan las principales conclusiones:

### **1. Distribución de Peligrosidad**

- La mayoría de los estados (representados en verde) tienen una clasificación de baja peligrosidad según el modelo.
- Algunos estados (amarillos y rojos) están clasificados como de peligrosidad media y alta, respectivamente.

### **2. Relación con Población Urbana**

- La población urbana, expresada como un porcentaje de la población total del estado, no muestra una relación clara con la clasificación de peligrosidad.
- Estados con baja población urbana pueden estar en clusters de alta peligrosidad y viceversa.

### **3. Características Criminales**

- Las características criminales normalizadas, como Asesinatos, Asaltos y Violaciones, muestran valores más altos en estados clasificados como de peligrosidad media y alta.

### **4. Limitaciones**

- La interpretación de la peligrosidad se basa en la clasificación del modelo y las características criminales normalizadas.
- La relación con la población urbana es compleja y no se puede establecer una conclusión clara.

## **Conclusión General**

El modelo proporciona una clasificación de peligrosidad que se relaciona principalmente con las características criminales normalizadas. Sin embargo, se requiere un análisis más detallado para comprender mejor la compleja relación entre la población urbana y la peligrosidad.
